In [1]:
from __future__ import print_function, division, absolute_import, unicode_literals
%load_ext tensorboard
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from PIL import ImageOps, Image
import glob
from sklearn.cluster import KMeans
from time import time
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Lambda, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
print('setup finished')


setup finished


In [3]:
pwd

'E:\\wall_segmentation\\anitha\\patch_making'

In [5]:
im_width, im_height = 256 , 256
path_train = 'E:\\wall_segmentation\\anitha\\patch_making'
#classes = ['dark_dense_distribution', 'Elongated_nuclei','light_dense_istribution','light_distribution',
 #          'No_nuclei_with_uniform_texture','No_nuclei_with_unstructured','Red_texture',  'Round_nuclei','Mixed']

In [6]:
path_train + '\\images'

'E:\\wall_segmentation\\anitha\\patch_making\\images'

In [7]:
def patch_maker(savedir, path, filename, target_size=(256,256)):
    
    '''opens one images at a time and saves them into patches of given hight and width. 
    It also handels RGBA format issues'''
    if not os.path.isdir(savedir):
            os.makedirs(savedir)
    img = Image.open(path + filename)
    width, height = img.size

    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size

    new_name = filename.split('.')[0]
    frame_num = 1
    for col_i in tqdm_notebook(range(0, width, w)):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            save_to= os.path.join(savedir, new_name + "_{:04}.jpg")
            if crop.mode in ('RGBA', 'LA'):
                background = Image.new(crop.mode[:-1], crop.size, (255,255,255))
                background.paste(crop, crop.split()[-1])
                crop = background
            crop.save(save_to.format(frame_num))
            frame_num += 1

In [8]:
def save_patches(savedir, path, images, masks):
    if not os.path.isdir(savedir):
            os.makedirs(savedir)
    path_im = path + '\\images\\'
    path_ms = path + '\\masks\\' 
    ids = next(os.walk(path_ms))[2]
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        '''image pathing'''
        print(id_.replace('.png', ''))
        patch_maker(savedir + 'images/images', path_im, id_.replace('.png', '.tif'))
        
        '''mask patching'''
        patch_maker(savedir + 'masks/masks', path_ms, id_)

In [15]:
save_patches(path_train,path_train, 'Images/', 'Masks/')

S340 (32)


In [15]:
'''filtering out the white background patchs
and filtering out the black patchs produced during patch making fillers'''


image_path= path_train+'\\images'
out_path = path_train+'\\image'
masks_path=path_train+'\\masks'

# a=0
#create output path for extracting not white patchs; if not exist 
if not os.path.exists(out_path):
    os.makedirs(out_path)

#make the list of patchs
image_files_list = glob.glob(image_path + '\\*jpg')
mask_files_list =glob.glob(masks_path+'\\*jpg')
output_files_list=glob.glob(out_path+'\\*jpg')

#function to filter out the white&black patchs and save in the output path
for image_file in image_files_list:
    filename = image_file.split('\\')[-1]
    img = Image.open(image_file)
    img = np.array(img)
    print(filename,':',img.mean())
    
    if img.mean() > 210:
#         print(img)
#         a=a+1
#         print(a)
        continue

    else:
        if img.mean() < 30:
            continue
        img = Image.fromarray(img)
        img.save(out_path+'\\'+ filename)
    

S340 (32)_0001.jpg : 220.11297607421875
S340 (32)_0002.jpg : 220.0811971028646
S340 (32)_0003.jpg : 219.500244140625
S340 (32)_0004.jpg : 219.59051513671875
S340 (32)_0005.jpg : 219.73071797688803
S340 (32)_0006.jpg : 219.18059285481772
S340 (32)_0007.jpg : 219.10010274251303
S340 (32)_0008.jpg : 218.98806762695312
S340 (32)_0009.jpg : 218.8064422607422
S340 (32)_0010.jpg : 218.54706827799478
S340 (32)_0011.jpg : 218.0869598388672
S340 (32)_0012.jpg : 217.97098795572916
S340 (32)_0013.jpg : 218.54627482096353
S340 (32)_0014.jpg : 218.7415008544922
S340 (32)_0015.jpg : 218.25487263997397
S340 (32)_0016.jpg : 218.89236958821616
S340 (32)_0017.jpg : 218.97880045572916
S340 (32)_0018.jpg : 217.61133829752603
S340 (32)_0019.jpg : 215.97108459472656
S340 (32)_0020.jpg : 213.01684061686197
S340 (32)_0021.jpg : 216.5614267985026
S340 (32)_0022.jpg : 20.315460205078125
S340 (32)_0023.jpg : 220.16204833984375
S340 (32)_0024.jpg : 220.29953002929688
S340 (32)_0025.jpg : 219.03594462076822
S340 (3

S340 (32)_0207.jpg : 214.7091318766276
S340 (32)_0208.jpg : 214.5464833577474
S340 (32)_0209.jpg : 217.27556355794272
S340 (32)_0210.jpg : 217.18414815266928
S340 (32)_0211.jpg : 217.3998057047526
S340 (32)_0212.jpg : 217.15522257486978
S340 (32)_0213.jpg : 215.78020222981772
S340 (32)_0214.jpg : 216.9002685546875
S340 (32)_0215.jpg : 218.135498046875
S340 (32)_0216.jpg : 217.12962849934897
S340 (32)_0217.jpg : 217.88313802083334
S340 (32)_0218.jpg : 217.78023274739584
S340 (32)_0219.jpg : 218.51056925455728
S340 (32)_0220.jpg : 20.62554931640625
S340 (32)_0221.jpg : 218.28199259440103
S340 (32)_0222.jpg : 216.78437296549478
S340 (32)_0223.jpg : 214.93172709147134
S340 (32)_0224.jpg : 202.98504130045572
S340 (32)_0225.jpg : 190.09750366210938
S340 (32)_0226.jpg : 171.49532063802084
S340 (32)_0227.jpg : 170.67125956217447
S340 (32)_0228.jpg : 170.14958699544272
S340 (32)_0229.jpg : 185.79212443033853
S340 (32)_0230.jpg : 202.39657592773438
S340 (32)_0231.jpg : 215.75806681315103
S340 (3

S340 (32)_0413.jpg : 215.3792266845703
S340 (32)_0414.jpg : 194.3123779296875
S340 (32)_0415.jpg : 168.22250366210938
S340 (32)_0416.jpg : 166.57986958821616
S340 (32)_0417.jpg : 199.74271138509116
S340 (32)_0418.jpg : 20.30639139811198
S340 (32)_0419.jpg : 217.8404083251953
S340 (32)_0420.jpg : 201.9429931640625
S340 (32)_0421.jpg : 188.29683939615884
S340 (32)_0422.jpg : 187.94780476888022
S340 (32)_0423.jpg : 171.34057108561197
S340 (32)_0424.jpg : 167.99405415852866
S340 (32)_0425.jpg : 178.56497192382812
S340 (32)_0426.jpg : 192.35352071126303
S340 (32)_0427.jpg : 202.51285807291666
S340 (32)_0428.jpg : 212.75176493326822
S340 (32)_0429.jpg : 215.60101826985678
S340 (32)_0430.jpg : 215.8282725016276
S340 (32)_0431.jpg : 216.29247029622397
S340 (32)_0432.jpg : 216.45518493652344
S340 (32)_0433.jpg : 216.18012491861978
S340 (32)_0434.jpg : 216.18115743001303
S340 (32)_0435.jpg : 203.9429168701172
S340 (32)_0436.jpg : 173.86874389648438
S340 (32)_0437.jpg : 164.6458485921224
S340 (32

S340 (32)_0619.jpg : 217.45907592773438
S340 (32)_0620.jpg : 217.95556640625
S340 (32)_0621.jpg : 218.71973164876303
S340 (32)_0622.jpg : 218.4742177327474
S340 (32)_0623.jpg : 217.62977600097656
S340 (32)_0624.jpg : 217.55735778808594
S340 (32)_0625.jpg : 208.67837524414062
S340 (32)_0626.jpg : 188.1088104248047
S340 (32)_0627.jpg : 180.67524210611978
S340 (32)_0628.jpg : 179.51226298014322
S340 (32)_0629.jpg : 177.8730926513672
S340 (32)_0630.jpg : 177.8387654622396
S340 (32)_0631.jpg : 197.02474466959634
S340 (32)_0632.jpg : 203.72865295410156
S340 (32)_0633.jpg : 214.9283243815104
S340 (32)_0634.jpg : 215.50200398763022
S340 (32)_0635.jpg : 215.7551015218099
S340 (32)_0636.jpg : 216.05907185872397
S340 (32)_0637.jpg : 216.1261240641276
S340 (32)_0638.jpg : 20.263809204101562
S340 (32)_0639.jpg : 219.68190002441406
S340 (32)_0640.jpg : 219.90739949544272
S340 (32)_0641.jpg : 219.3314463297526
S340 (32)_0642.jpg : 219.59849548339844
S340 (32)_0643.jpg : 220.0520273844401
S340 (32)_06

In [16]:
#create mask and filtered image list
mask_list=[]
out_list=[]

for out_file in output_files_list:
    image = out_file.split('\\')[-1]
    out_list.append(image)
    
    
for mask_file in mask_files_list:
    mask = mask_file.split('\\')[-1]
    mask_list.append(mask)

#crete a delete list from masks(filtering the masks according to filtered image list)
def intersection(list1,list2):
    return set(list2).difference(list1)
    
delete_list=[]
delete_list=intersection(out_list,mask_list)
#print(delete_list)
#print(len(delete_list))
#print(len(out_list))

In [12]:
#delete the masks files
for file in delete_list:
    path=os.path.join(masks_path,file)
    print(path)
    os.remove(path)

In [18]:
#check the number of files in the given path
print(masks_path)
list = os.listdir(masks_path) 
number_files = len(list)
print(number_files)    

E:\wall_segmentation\anitha\patch_making\masks
278
